In [1]:
import sys
from pathlib import Path
import os 
sys.path.append('/home/gridsan/vyuan/.local/lib/python3.9/site-packages/')

current_path = Path.cwd()

# Go to top of the root and append
root = current_path.parents[1]
sys.path.append(str(root))

os.chdir(root)

In [2]:
import pandas as pd
import math

In [3]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com


In [4]:
import openpyxl

Excel avec 
- Les 9 files de base (6 cbm, 3 excel) 
- Un file avec la différence d'accuracy entre male et female pour chacun des groupes: en row les 9 datasets, en ordonnée "différence entre male et female du groupe 1", "du groupe 2", "total"
- 3 file, un pour balanced, imbalanced 1, imbalanced 2 pour voir les différences entre les groupes

In [33]:
# Read files

path_dict = {
    'Baseline Balanced': 'results/imSitu/30_verbs/baseline/balanced/accuracies_by_group.csv',
    'Baseline Imbalanced 1': 'results/imSitu/30_verbs/baseline/imbalanced_1/accuracies_by_group.csv',
    'Baseline Imbalanced 2': 'results/imSitu/30_verbs/baseline/imbalanced_2/accuracies_by_group.csv',
    
    'CBM Dense Balanced': 'results/imSitu/30_verbs/CBM/no_gender/balanced_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 1': 'results/imSitu/30_verbs/CBM/no_gender/imbalanced_1_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 2': 'results/imSitu/30_verbs/CBM/no_gender/imbalanced_2_dense/accuracies_by_group.csv',
    
    'CBM Sparse Balanced': 'results/imSitu/30_verbs/CBM/no_gender/balanced_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 1': 'results/imSitu/30_verbs/CBM/no_gender/imbalanced_1_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 2': 'results/imSitu/30_verbs/CBM/no_gender/imbalanced_2_sparse/accuracies_by_group.csv',
    
    'CBM Dense Balanced Gender': 'results/imSitu/30_verbs/CBM/gender/balanced_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 1 Gender': 'results/imSitu/30_verbs/CBM/gender/imbalanced_1_dense/accuracies_by_group.csv',
    'CBM Dense Imbalanced 2 Gender': 'results/imSitu/30_verbs/CBM/gender/imbalanced_2_dense/accuracies_by_group.csv',
    
    'CBM Sparse Balanced Gender': 'results/imSitu/30_verbs/CBM/gender/balanced_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 1 Gender': 'results/imSitu/30_verbs/CBM/gender/imbalanced_1_sparse/accuracies_by_group.csv',
    'CBM Sparse Imbalanced 2 Gender': 'results/imSitu/30_verbs/CBM/gender/imbalanced_2_sparse/accuracies_by_group.csv',
        
    'Baseline full': 'results/imSitu/30_verbs_full/baseline/accuracies_by_group.csv',
    'CBM Sparse full': 'results/imSitu/30_verbs_full/CBM/sparse_no_gender',
    'CBM Dense full': 'results/imSitu/30_verbs_full/CBM/dense_no_gender',
    'CBM Sparse full': 'results/imSitu/30_verbs_full/CBM/sparse_gender',
    'CBM Dense full': 'results/imSitu/30_verbs_full/CBM/dense_gender',
}

path_dict = {
    'Baseline full': 'results/imSitu/30_verbs_full/baseline/accuracies_by_group.csv',
    'CBM Sparse full': 'results/imSitu/30_verbs_full/CBM/sparse_no_gender/accuracies_by_group.csv',
    'CBM Dense full': 'results/imSitu/30_verbs_full/CBM/dense_no_gender/accuracies_by_group.csv',
    'CBM Sparse full gender': 'results/imSitu/30_verbs_full/CBM/sparse_gender/accuracies_by_group.csv',
    'CBM Dense full gender': 'results/imSitu/30_verbs_full/CBM/dense_gender/accuracies_by_group.csv',
}

In [34]:
result_dict = {}

for name, path in path_dict.items():
    result_dict[name] = pd.read_csv(path, index_col=0)

In [35]:
results_temp = []

for name_df,df in result_dict.items():
    #if name_df in ['Baseline Imbalanced 1', 'Baseline Imbalanced 2', 'Baseline Balanced', 'Baseline full']:
    if name_df == 'Baseline full':
            male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
            male_minus_female_1 = 100 * (male_1 - female_1)
            male_minus_female_2 = 100 * (male_2 - female_2)

            accuracy_parity_1 = abs(male_minus_female_1)
            accuracy_parity_2 = abs(male_minus_female_2)

            highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
            highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'

            average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
            average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
            
            result_temp_df = pd.DataFrame({'Experiment': [name_df], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                             })
            
            results_temp.append(result_temp_df)
            
final_temp_df = pd.concat(results_temp).set_index('Experiment')

In [36]:
final_temp_df

,Average Accuracy,Average Accuracy 1,Average Accuracy 2,Accuracy Parity 1,Accuracy Parity 2,Highest accuracy 1,Highest accuracy 2
Experiment,,,,,,,
Baseline full,42.162162,40.502355,43.768997,4.47036,4.295488,male,male


In [37]:
results = []

for name, df in result_dict.items():
    male_1, female_1, male_2, female_2 = df.loc['male_1', 'accuracy'], df.loc['female_1', 'accuracy'], df.loc['male_2', 'accuracy'], df.loc['female_2', 'accuracy']
    
    male_minus_female_1 = 100 * (male_1 - female_1)
    male_minus_female_2 = 100 * (male_2 - female_2)
    
    accuracy_parity_1 = abs(male_minus_female_1)
    accuracy_parity_2 = abs(male_minus_female_2)
    
    highest_accuracy_1 = 'male' if male_minus_female_1 > 0 else 'female'
    highest_accuracy_2 = 'male' if male_minus_female_2 > 0 else 'female'
    
    average_accuracy_1 = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'])
    average_accuracy_2 = 100 * (df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
    average_accuracy = 100 * (df.loc['male_1', 'correct'] + df.loc['female_1', 'correct'] + df.loc['male_2', 'correct'] + df.loc['female_2', 'correct']) / (df.loc['male_1', 'count'] + df.loc['female_1', 'count'] + df.loc['male_2', 'count'] + df.loc['female_2', 'count'])
    
    if '1' in name:
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Imbalanced 1', 'Average Accuracy'])
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Imbalanced 1', 'Accuracy Parity 1'])
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Imbalanced 1', 'Accuracy Parity 2'])
        
    elif '2' in name:
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Imbalanced 2', 'Average Accuracy']) 
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Imbalanced 2', 'Accuracy Parity 1']) 
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Imbalanced 2', 'Accuracy Parity 2'])
    elif 'full' in name:
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline full', 'Average Accuracy']) 
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline full', 'Accuracy Parity 1']) 
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline full', 'Accuracy Parity 2'])
    else: 
        percentage_accuracy = (average_accuracy - final_temp_df.loc['Baseline Balanced', 'Average Accuracy']) 
        percentage_ap_1 = (accuracy_parity_1 - final_temp_df.loc['Baseline Balanced', 'Accuracy Parity 1']) 
        percentage_ap_2 = (accuracy_parity_2 - final_temp_df.loc['Baseline Balanced', 'Accuracy Parity 2']) 
    
    
    result_df = pd.DataFrame({'Experiment': [name], 
                              'Average Accuracy': [average_accuracy],
                              'Average Accuracy 1': [average_accuracy_1],
                              'Average Accuracy 2': [average_accuracy_2],
                              'Accuracy Parity 1': [accuracy_parity_1], 
                              'Accuracy Parity 2': [accuracy_parity_2],
                              'Highest accuracy 1': [highest_accuracy_1],
                              'Highest accuracy 2': [highest_accuracy_2],
                              'Accuracy change to Baseline': [percentage_accuracy],
                              'Accuracy Parity 1 change to Baseline': [percentage_ap_1],
                              'Accuracy Parity 2 change to Baseline': [percentage_ap_2],
                             })
    results.append(result_df)
    
final_df = pd.concat(results).reset_index(drop=True)

In [38]:
df_balanced = final_df[final_df['Experiment'].isin(['Baseline Balanced', 'CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender'])]
df_imbalanced_1 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 1', 'CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender'])]
df_imbalanced_2 = final_df[final_df['Experiment'].isin(['Baseline Imbalanced 2', 'CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender'])]
df_full = final_df[final_df['Experiment'].isin(['Baseline full', 'CBM Sparse full', 'CBM Dense full', 'CBM Sparse full gender', 'CBM Dense full gender'])]

In [39]:
result_dict['Comparison Metrics'] = final_df
result_dict['Comparison Metrics Balanced'] = df_balanced
result_dict['Comparison Metrics Imbalanced 1'] = df_imbalanced_1
result_dict['Comparison Metrics Imbalanced 2'] = df_imbalanced_2
result_dict['Comparison_Metrics full'] = df_full

In [40]:
with pd.ExcelWriter(Path.cwd() / 'results' / 'imSitu' / '30_verbs_full' / 'results_full.xlsx') as writer:
    for name, df in result_dict.items():
        df.to_excel(writer, sheet_name=name, index=False)

## Data Analysis

In [28]:
df = result_dict['Comparison Metrics']

In [29]:
df

,Experiment,Average Accuracy,Average Accuracy 1,Average Accuracy 2,Accuracy Parity 1,Accuracy Parity 2,Highest accuracy 1,Highest accuracy 2,Accuracy change to Baseline,Accuracy Parity 1 change to Baseline,Accuracy Parity 2 change to Baseline
0,Baseline Balanced,34.741379,32.817869,36.678201,6.529210,1.384083,male,male,0.000000,0.000000,0.000000
1,Baseline Imbalanced 1,33.879310,35.051546,32.698962,17.182131,18.339100,male,female,0.000000,0.000000,0.000000
2,Baseline Imbalanced 2,34.051724,36.082474,32.006920,14.432990,24.567474,female,male,0.000000,0.000000,0.000000
3,CBM Dense Balanced,38.189655,38.144330,38.235294,4.123711,0.346021,male,male,3.448276,-2.405498,-1.038062
4,CBM Dense Imbalanced 1,34.827586,34.364261,35.294118,17.182131,20.761246,male,female,0.948276,0.000000,2.422145
5,CBM Dense Imbalanced 2,36.810345,38.831615,34.775087,8.934708,18.339100,female,male,2.758621,-5.498282,-6.228374
6,CBM Sparse Balanced,34.741379,32.817869,36.678201,6.529210,1.384083,male,male,0.000000,0.000000,0.000000
7,CBM Sparse Imbalanced 1,34.051724,33.848797,34.256055,16.838488,13.840830,male,female,0.172414,-0.343643,-4.498270
8,CBM Sparse Imbalanced 2,36.206897,39.690722,32.698962,11.340206,15.570934,female,male,2.155172,-3.092784,-8.996540
9,CBM Dense Balanced Gender,38.017241,38.144330,37.889273,4.123711,0.346021,male,male,3.275862,-2.405498,-1.038062


In [15]:
metrics = ['Accuracy change to Baseline	', 'Accuracy Parity 1 change to Baseline', 'Accuracy Parity 2 change to Baseline']

balanced = ['CBM Dense Balanced', 'CBM Sparse Balanced', 'CBM Dense Balanced Gender', 'CBM Sparse Balanced Gender']
imbalanced_1 = ['CBM Dense Imbalanced 1', 'CBM Sparse Imbalanced 1', 'CBM Dense Imbalanced 1 Gender', 'CBM Sparse Imbalanced 1 Gender']
imbalanced_2 = ['CBM Dense Imbalanced 2', 'CBM Sparse Imbalanced 2', 'CBM Dense Imbalanced 2 Gender', 'CBM Sparse Imbalanced 2 Gender']

In [ ]:
result = {}


for experiment in balanced:
    result_experiment = {'Experiment': experiment}
    for metric in metrics:
        baseline_value = df.loc['Baseline Balanced', metric]
        experiment_value = df.loc[experiment, metric]
        percentage_change = (baseline_value - experiment_value) / (baseline_value)
        result_experiment[experiment] 